<a href="https://colab.research.google.com/github/eschmidt39/Homework-4/blob/main/Homework4_Schmidt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm
from scipy.optimize import minimize
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [29]:
filedir = '/content/drive/MyDrive/FWE458 - Envir. Data Sci./Data_Sets/'
fname = filedir+'Wcr_GPPdaily.csv'

df = pd.read_csv(fname)
print(df)

      TIMESTAMP    TA_F  SW_IN_F  VPD_F  GPP_NT_VUT_REF
0      19990101 -20.063   72.603  0.413       -0.517364
1      19990102 -12.814   12.358  0.147       -0.094241
2      19990103 -12.625   33.132  0.128       -0.166819
3      19990104 -18.652   93.481  0.263       -0.582301
4      19990105 -20.269   45.502  0.261       -0.568240
...         ...     ...      ...    ...             ...
5839   20141227  -5.112   22.956  0.497        0.195777
5840   20141228 -12.408   33.332  0.531        0.093025
5841   20141229 -18.204   69.597  0.461       -0.234488
5842   20141230 -22.252   81.959  0.448       -0.315846
5843   20141231 -19.436   42.164  0.619       -0.217537

[5844 rows x 5 columns]


In [30]:
#drop TIMESTAMP column
df.drop(columns=["TIMESTAMP"], inplace=True)
print(df)

        TA_F  SW_IN_F  VPD_F  GPP_NT_VUT_REF
0    -20.063   72.603  0.413       -0.517364
1    -12.814   12.358  0.147       -0.094241
2    -12.625   33.132  0.128       -0.166819
3    -18.652   93.481  0.263       -0.582301
4    -20.269   45.502  0.261       -0.568240
...      ...      ...    ...             ...
5839  -5.112   22.956  0.497        0.195777
5840 -12.408   33.332  0.531        0.093025
5841 -18.204   69.597  0.461       -0.234488
5842 -22.252   81.959  0.448       -0.315846
5843 -19.436   42.164  0.619       -0.217537

[5844 rows x 4 columns]


In [31]:
#rename all columns, added 'inplace = True' to rename within the current df rather than make a copy with the new names
df.rename(columns={'TA_F':'TA', 'SW_IN_F':'SW', 'VPD_F':'VPD', 'GPP_NT_VUT_REF':'GPP'}, inplace=True)

TA = df['TA']
SW = df['SW']
VPD = df['VPD']
GPP = df['GPP']

print(df)

          TA      SW    VPD       GPP
0    -20.063  72.603  0.413 -0.517364
1    -12.814  12.358  0.147 -0.094241
2    -12.625  33.132  0.128 -0.166819
3    -18.652  93.481  0.263 -0.582301
4    -20.269  45.502  0.261 -0.568240
...      ...     ...    ...       ...
5839  -5.112  22.956  0.497  0.195777
5840 -12.408  33.332  0.531  0.093025
5841 -18.204  69.597  0.461 -0.234488
5842 -22.252  81.959  0.448 -0.315846
5843 -19.436  42.164  0.619 -0.217537

[5844 rows x 4 columns]


In [32]:
#check for missing values and drop them, filter out missing values
ft = (GPP > -999) & (TA > 10) & (VPD < 5)
df = df[ft]

#same number or rows means no missing values
print(df)

          TA       SW    VPD       GPP
89    13.069  179.065  4.413  0.218743
90    10.363   91.985  2.293 -0.224844
91    10.160  131.276  1.998 -0.150231
124   14.929   64.452  4.208 -0.335365
125   13.003   48.432  1.125 -2.432490
...      ...      ...    ...       ...
5740  14.816  127.042  4.984  4.512060
5743  11.830  207.858  4.653  3.990750
5746  15.621  107.746  4.745  2.035560
5753  12.711   47.479  1.637  0.553575
5775  10.537  110.491  3.139  1.007660

[1245 rows x 4 columns]


In [33]:
#create interaction terms and add as a new column

#SWxVPD
df["SW x VPD"] = df['SW'] * df['VPD']
#SWxTA
df["SW x TA"] = df['SW'] * df['TA']
#VPDxTA
df["VPD x TA"] = df['VPD'] * df['TA']

SWxVPD = df["SW x VPD"]
SWxTA = df["SW x TA"]
VPDxTA = df['VPD x TA']

print(df)

          TA       SW    VPD       GPP    SW x VPD      SW x TA   VPD x TA
89    13.069  179.065  4.413  0.218743  790.213845  2340.200485  57.673497
90    10.363   91.985  2.293 -0.224844  210.921605   953.240555  23.762359
91    10.160  131.276  1.998 -0.150231  262.289448  1333.764160  20.299680
124   14.929   64.452  4.208 -0.335365  271.214016   962.203908  62.821232
125   13.003   48.432  1.125 -2.432490   54.486000   629.761296  14.628375
...      ...      ...    ...       ...         ...          ...        ...
5740  14.816  127.042  4.984  4.512060  633.177328  1882.254272  73.842944
5743  11.830  207.858  4.653  3.990750  967.163274  2458.960140  55.044990
5746  15.621  107.746  4.745  2.035560  511.254770  1683.100266  74.121645
5753  12.711   47.479  1.637  0.553575   77.723123   603.505569  20.807907
5775  10.537  110.491  3.139  1.007660  346.831249  1164.243667  33.075643

[1245 rows x 7 columns]


In [34]:
#linear regression model using scipy optimization

def costfunc(param, SW, VPD, TA, GPP, SWxVPD, SWxTA, VPDxTA):
  β0, β1, β2, β3, β4, β5, β6 = param
  GPPpred = β0+β1*SW+β2*VPD+β3*TA+β4*(SWxVPD)+β5*(SWxTA)+β6*(VPDxTA)
  #mean squared error
  diff = np.mean((GPP - GPPpred)**2)
  return diff

guess = [10, 10, 10, 10, 10, 10, 10]
mymin = minimize(costfunc, guess, args=(SW, VPD, TA, GPP, SWxVPD, SWxTA, VPDxTA))
β0, β1, β2, β3, β4, β5, β6 = mymin.x

mymin

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 12.89769792525038
        x: [-2.155e+00  5.277e-02 -1.731e+00  1.682e-01 -5.000e-03
            -2.305e-04  1.054e-01]
      nit: 24
      jac: [-2.623e-06 -5.989e-04 -9.060e-06 -4.601e-05 -2.176e-03
            -1.070e-02 -1.659e-04]
 hess_inv: [[ 2.096e-07 -1.296e-09 ...  5.164e-11 -1.415e-09]
            [-1.296e-09  4.604e-03 ... -2.097e-04  1.688e-02]
            ...
            [ 5.164e-11 -2.097e-04 ...  1.287e-05 -8.068e-04]
            [-1.415e-09  1.688e-02 ... -8.068e-04  6.751e-02]]
     nfev: 328
     njev: 41

In [35]:
#linear regression model using sklearn
x = df[['TA', 'SW', 'VPD', 'SW x TA', 'SW x VPD', 'VPD x TA']]
y = df['GPP']

model = LinearRegression()
model.fit(x, y)

print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)

Coefficients: [ 1.68068185e-01  5.27436999e-02 -1.72912770e+00 -2.28434586e-04
 -5.00055258e-03  1.05349204e-01]
Intercept: -2.1531868633135485


I couldn't calculate R2 for scipy optimization model, so the sklearn would be used to compare all possible factors (SW, TA, VPD) - it's also easier to do and has documentation that I was able to understand better, but without being able to compare R2 between the two models makes it unable to determine which is more applicable

In [57]:
#compare R2 for both models
#scipy optimization model

x1 = df[['TA', 'SW', 'VPD']]
y1 = df['GPP']
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

print(f"R-squared={r_value**2}")

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 6 and the array at index 1 has size 1245

In [50]:
#sklearn model R2
GPP_pred = model.predict(x)
sklearn_r2 = r2_score(df['GPP'], GPP_pred)
print('sklearn Model R2 Score:', sklearn_r2)

sklearn Model R2 Score: 0.4074362266544902


based on the sklearn model, only 40% of the data can be explained with the model (60%) is due to another reason, whether this is an acceptable R2 depends on the research question posed and other surrounding context